In [1]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE,RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from itertools import combinations 
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
import copy 

Using TensorFlow backend.
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtype

CPU times: user 2.39 s, sys: 761 ms, total: 3.15 s
Wall time: 5.9 s


In [2]:
df=pd.read_csv('60s_window_wrist_chest.csv',index_col=0)
df=df[df['label']<2]

In [3]:
features=df.columns.tolist()
features

removed = ['label']
for rem in removed:
    features.remove(rem)

features_with_sub=[]
features_with_sub[:]=features
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)

sm = SMOTE(random_state=2)
X, y= sm.fit_sample(df[features_with_sub], df['label'])
df_new=pd.concat([pd.DataFrame(X,columns=features_with_sub),pd.DataFrame(y,columns=['label'])],axis=1)
df_new

for i in range (len(list(df_new['subject']))):
    df_new['subject'][i] = min([2,3,4,5,6,7,8,9,10,11,13,14,15,16,17], key=lambda x:abs(x-df_new['subject'][i]))
df_new['subject']=df_new['subject'].astype(int)

p_d=pd.read_csv('personal_detail.csv',index_col=0)

df_new_1=df_new.merge(p_d,on='subject')
df_new_1

70


/home/sf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


net_acc_mean  net_acc_std  net_acc_min  net_acc_max  ACC_x_mean  \
0        0.025961     0.013811     0.000000     0.087383    0.023431   
1        0.027640     0.010597     0.002752     0.054356    0.027640   
2        0.028389     0.006937     0.000000     0.066053    0.028378   
3        0.033268     0.007670     0.000000     0.074998    0.032960   
4        0.037021     0.001284     0.027522     0.043347    0.037021   
..            ...          ...          ...          ...         ...   
619      0.031629     0.001143     0.028210     0.039219   -0.031629   
620      0.033437     0.001540     0.013073     0.051604   -0.033437   
621      0.031757     0.001600     0.013761     0.037155   -0.031757   
622      0.029441     0.002093     0.020770     0.054100   -0.004075   
623      0.029484     0.002074     0.020918     0.053804   -0.004624   

     ACC_x_std  ACC_x_min  ACC_x_max  ACC_y_mean     ACC_y_std  ...  label  \
0     0.017769  -0.037843   0.087383    0.000016  1.222594e-05  ...      0   
1     0.010597   0.002752   0.054356    0.000019  7.290999e-06  ...      0   
2     0.006985  -0.002752   0.066053    0.000020  4.805734e-06  ...      0   
3     0.008904  -0.030962   0.074998    0.000023  6.126303e-06  ...      0   
4     0.001284   0.027522   0.043347    0.000025  8.837530e-07  ...      0   
..         ...        ...        ...         ...           ...  ...    ...   
619   0.001143  -0.039219  -0.028210   -0.000022  7.864926e-07  ...      1   
620   0.001540  -0.051604  -0.013073   -0.000023  1.059291e-06  ...      1   
621   0.001600  -0.037155  -0.013761   -0.000022  1.101023e-06  ...      1   
622   0.002093  -0.014956   0.018375   -0.000003  1.440095e-06  ...      1   
623   0.002074  -0.015439   0.017447   -0.000003  1.427082e-06  ...      1   

     age  height  weight  gender_ female  coffee_today_YES  sport_today_YES  \
0     27     175      80               0                 0                0   
1     27     175      80               0                 0                0   
2     27     175      80               0                 0                0   
3     27     175      80               0                 0                0   
4     27     175      80               0                 0                0   
..   ...     ...     ...             ...               ...              ...   
619   29     165      55               1                 0                0   
620   29     165      55               1                 0                0   
621   29     165      55               1                 0                0   
622   29     165      55               1                 0                0   
623   29     165      55               1                 0                0   

     smoker_YES  feel_ill_today_YES  bmi  
0             0                   0    1  
1             0                   0    1  
2             0                   0    1  
3             0                   0    1  
4             0                   0    1  
..          ...                 ...  ...  
619           0                   0    0  
620           0                   0    0  
621           0                   0    0  
622           0                   0    0  
623           0                   0    0  

[624 rows x 81 columns]

In [4]:
sel_fea = ['EDA_tonic_mean','EDA_tonic_max','EDA_phasic_max','ECG_std']

In [61]:
train=df_new_1[df_new_1['subject']<=9]
test=df_new_1[df_new_1['subject']>9]

#XG-BOOST#

In [47]:
from xgboost import XGBClassifier
import xgboost as xgb
xg_reg = xgb.XGBClassifier()
xg_reg.fit(train[sel_fea],train['label'])
y_pred=xg_reg.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.95      0.88      0.92       147
           1       0.90      0.96      0.93       161

    accuracy                           0.92       308
   macro avg       0.92      0.92      0.92       308
weighted avg       0.92      0.92      0.92       308



In [48]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    xg_reg.fit(train[sel_fea],train['label'])
    y_pred=xg_reg.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

10
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        21
           1       1.00      0.96      0.98        28

    accuracy                           0.98        49
   macro avg       0.98      0.98      0.98        49
weighted avg       0.98      0.98      0.98        49

11
              precision    recall  f1-score   support

           0       0.94      0.71      0.81        21
           1       0.80      0.96      0.87        25

    accuracy                           0.85        46
   macro avg       0.87      0.84      0.84        46
weighted avg       0.86      0.85      0.84        46

13
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        21
           1       1.00      0.93      0.97        30

    accuracy                           0.96        51
   macro avg       0.96      0.97      0.96        51
weighted avg       0.96      0.96      0.96        51

14
       

#GradientBoosting#

In [50]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(train[sel_fea],train['label'])
y_pred=gb.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.96      0.80      0.87       147
           1       0.84      0.97      0.90       161

    accuracy                           0.89       308
   macro avg       0.90      0.88      0.88       308
weighted avg       0.90      0.89      0.89       308



In [51]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    gb.fit(train[sel_fea],train['label'])
    y_pred=gb.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

10
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        21
           1       1.00      0.96      0.98        28

    accuracy                           0.98        49
   macro avg       0.98      0.98      0.98        49
weighted avg       0.98      0.98      0.98        49

11
              precision    recall  f1-score   support

           0       0.94      0.76      0.84        21
           1       0.83      0.96      0.89        25

    accuracy                           0.87        46
   macro avg       0.88      0.86      0.87        46
weighted avg       0.88      0.87      0.87        46

13
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        21
           1       1.00      0.93      0.97        30

    accuracy                           0.96        51
   macro avg       0.96      0.97      0.96        51
weighted avg       0.96      0.96      0.96        51

14
       

#Gaussian

In [53]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(train[sel_fea],train['label'])
y_pred=model.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       147
           1       0.87      0.88      0.87       161

    accuracy                           0.87       308
   macro avg       0.87      0.87      0.87       308
weighted avg       0.87      0.87      0.87       308



In [54]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    model.fit(train[sel_fea],train['label'])
    y_pred=model.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

10
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        21
           1       1.00      0.96      0.98        28

    accuracy                           0.98        49
   macro avg       0.98      0.98      0.98        49
weighted avg       0.98      0.98      0.98        49

11
              precision    recall  f1-score   support

           0       0.57      0.76      0.65        21
           1       0.72      0.52      0.60        25

    accuracy                           0.63        46
   macro avg       0.65      0.64      0.63        46
weighted avg       0.65      0.63      0.63        46

13
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        30

    accuracy                           1.00        51
   macro avg       1.00      1.00      1.00        51
weighted avg       1.00      1.00      1.00        51

14
       

#RANDOMFORREST

In [56]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50)
rf.fit(train[sel_fea],train['label'])
y_pred=rf.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       147
           1       0.91      0.93      0.92       161

    accuracy                           0.92       308
   macro avg       0.92      0.91      0.92       308
weighted avg       0.92      0.92      0.92       308



In [57]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    rf.fit(train[sel_fea],train['label'])
    y_pred=rf.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

10
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        21
           1       1.00      0.96      0.98        28

    accuracy                           0.98        49
   macro avg       0.98      0.98      0.98        49
weighted avg       0.98      0.98      0.98        49

11
              precision    recall  f1-score   support

           0       0.94      0.71      0.81        21
           1       0.80      0.96      0.87        25

    accuracy                           0.85        46
   macro avg       0.87      0.84      0.84        46
weighted avg       0.86      0.85      0.84        46

13
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        21
           1       1.00      0.97      0.98        30

    accuracy                           0.98        51
   macro avg       0.98      0.98      0.98        51
weighted avg       0.98      0.98      0.98        51

14
       

#DecisionTree

In [59]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train[sel_fea],train['label'])
y_pred=clf.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.85      0.75      0.79       147
           1       0.79      0.88      0.83       161

    accuracy                           0.81       308
   macro avg       0.82      0.81      0.81       308
weighted avg       0.82      0.81      0.81       308



In [60]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    clf.fit(train[sel_fea],train['label'])
    y_pred=clf.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

10
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        21
           1       1.00      0.86      0.92        28

    accuracy                           0.92        49
   macro avg       0.92      0.93      0.92        49
weighted avg       0.93      0.92      0.92        49

11
              precision    recall  f1-score   support

           0       0.83      0.71      0.77        21
           1       0.79      0.88      0.83        25

    accuracy                           0.80        46
   macro avg       0.81      0.80      0.80        46
weighted avg       0.81      0.80      0.80        46

13
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        21
           1       1.00      0.87      0.93        30

    accuracy                           0.92        51
   macro avg       0.92      0.93      0.92        51
weighted avg       0.93      0.92      0.92        51

14
       

#ExtraTrees

In [62]:
et = ExtraTreesClassifier(n_estimators=50,n_jobs=10,random_state=12)
et.fit(train[sel_fea],train['label'])
y_pred=et.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95       147
           1       0.92      0.99      0.96       161

    accuracy                           0.95       308
   macro avg       0.96      0.95      0.95       308
weighted avg       0.96      0.95      0.95       308



In [63]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    et.fit(train[sel_fea],train['label'])
    y_pred=et.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

10
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        28

    accuracy                           1.00        49
   macro avg       1.00      1.00      1.00        49
weighted avg       1.00      1.00      1.00        49

11
              precision    recall  f1-score   support

           0       1.00      0.76      0.86        21
           1       0.83      1.00      0.91        25

    accuracy                           0.89        46
   macro avg       0.92      0.88      0.89        46
weighted avg       0.91      0.89      0.89        46

13
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        30

    accuracy                           1.00        51
   macro avg       1.00      1.00      1.00        51
weighted avg       1.00      1.00      1.00        51

14
       